In [ ]:
# dataset : ../dataset/WorldCupShootouts_２.csv
#
# Zone : zone_num(1~9)
# Foot : R = 9 // L = 1
# Keeper : R = 9 // L = 1 // C = 5
#
# Goal : O = 1 // X = 0

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
df = pd.read_csv("../dataset/WorldCupShootouts_3.csv")
df

,Zone,Foot,Keeper,Goal
0,7,9,9,1
1,9,9,5,1
2,6,9,1,1
3,2,9,5,1
4,9,9,1,1
...,...,...,...,...
274,7,9,1,1
275,7,9,9,1
276,4,9,9,1
277,7,9,9,1


In [9]:
x = df.loc[:, df.columns != 'Goal'].values
x

array([[7, 9, 9],
       [9, 9, 5],
       [6, 9, 1],
       [2, 9, 5],
       [9, 9, 1],
       [4, 9, 1],
       [8, 1, 1],
       [3, 9, 9],
       [9, 9, 1],
       [9, 9, 5],
       [7, 9, 1],
       [9, 9, 5],
       [4, 9, 1],
       [2, 9, 9],
       [6, 9, 9],
       [6, 9, 9],
       [8, 9, 1],
       [9, 1, 9],
       [4, 1, 9],
       [1, 9, 9],
       [7, 9, 1],
       [7, 9, 9],
       [1, 1, 9],
       [6, 1, 1],
       [8, 1, 1],
       [8, 9, 1],
       [7, 9, 1],
       [7, 9, 1],
       [4, 9, 1],
       [7, 9, 9],
       [9, 9, 1],
       [7, 9, 1],
       [4, 9, 9],
       [1, 9, 1],
       [7, 9, 9],
       [1, 9, 1],
       [5, 1, 9],
       [1, 9, 9],
       [5, 9, 1],
       [3, 1, 9],
       [2, 1, 9],
       [9, 9, 9],
       [9, 9, 1],
       [4, 1, 1],
       [7, 1, 9],
       [1, 9, 1],
       [8, 1, 1],
       [7, 9, 1],
       [6, 9, 1],
       [6, 9, 1],
       [1, 9, 9],
       [1, 9, 9],
       [4, 9, 9],
       [7, 1, 1],
       [7, 1, 9],
       [3,

In [10]:
y = df['Goal'].values
y

array([1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1], dtype=int64)

In [11]:
print(x[0])
print(y[0])

[7 9 9]
1


In [12]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

print(x_train.shape)
print(y_train.shape)

print(x_test.shape)
print(y_test.shape)

(223, 3)
(223,)
(56, 3)
(56,)


In [41]:
from torch.utils.data import DataLoader, Dataset, TensorDataset

class custom_data(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def norm(self, x, m = (5.595, 7.394, 4.785), std=(1, 1, 1)):
        return (x-m)/std

    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        x = self.x[index]
        y = self.y[index]

        x = self.norm(x)

        x = torch.tensor(x, dtype = torch.float32)
        y = torch.tensor(y, dtype = torch.float32).view(1)

        return x, y
    

In [42]:
train_dataset = custom_data(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True)

test_dataset = custom_data(x_test, y_test)
test_loader = DataLoader(test_dataset, batch_size = 64, shuffle = True)

In [43]:
x_in, y_out = next(iter(train_loader))

print(x_in.shape)
print(y_out.shape)

print(x_in.dtype)
print(y_out.dtype)

torch.Size([64, 3])
torch.Size([64, 1])
torch.float32
torch.float32


In [44]:
class Mymodel(nn.Module):
    def __init__(self):
        super(Mymodel, self).__init__()
        self.f1 = nn.Linear(3, 100)
        self.f2 = nn.Linear(100, 1)
        self.act = nn.Sigmoid()

    def forward(self, x):              # x = input
        out = self.f1(x)
        out = self.act(out)
        out = self.f2(out)
        out = self.act(out)

        return out

model = Mymodel()
        

In [45]:
from torchsummary import summary
summary(model, (1, 3), device='CPU')


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 1, 100]             400
           Sigmoid-2               [-1, 1, 100]               0
            Linear-3                 [-1, 1, 1]             101
           Sigmoid-4                 [-1, 1, 1]               0
Total params: 501
Trainable params: 501
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [46]:
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr = 0.001)

In [47]:
#from tqdm import tqdm

num_epochs = 1000

for epoch in range(num_epochs):
    for x_batch, y_batch in train_loader:
        outputs = model(x_batch)             # 1. prediction

        optimizer.zero_grad()                # 2. init optim
        loss = criterion(outputs, y_batch)   # 3. calculate loss
        loss.backward()                      # 4. calculate gradient
        optimizer.step()                     # 5. update the model

    if (epoch+1) % 100 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

Epoch 100, Loss: 0.6740021705627441
Epoch 200, Loss: 0.669919490814209
Epoch 300, Loss: 0.6514578461647034
Epoch 400, Loss: 0.5716912746429443
Epoch 500, Loss: 0.6504131555557251
Epoch 600, Loss: 0.6227173805236816
Epoch 700, Loss: 0.73459792137146
Epoch 800, Loss: 0.70462965965271
Epoch 900, Loss: 0.5804933905601501
Epoch 1000, Loss: 0.6002780199050903


In [67]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for x_batch, y_batch in test_loader:
        outputs = model(x_batch)
         
        _, predicted = torch.max(outputs.data, 1)
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()      # predict와 y_batch가 같을때 적중률

print(f'Accuracy : {100 * (correct/len(predicted)) / total}%')

Accuracy : 30.357142857142858%
